# Introduction

This notebook is intended to experiment with Sentence Similarity techniques.

## BERT

In [2]:
# Import Standard Libraries
from transformers import AutoTokenizer, AutoModel
import torch

In [3]:
# Define model
model_name = 'sentence-transformers/bert-base-nli-mean-tokens'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

config.json: 100%|██████████| 625/625 [00:00<00:00, 3.22MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 1.02MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.53MB/s]
added_tokens.json: 100%|██████████| 2.00/2.00 [00:00<00:00, 5.46kB/s]
special_tokens_map.json: 100%|██████████| 112/112 [00:00<00:00, 498kB/s]
pytorch_model.bin: 100%|██████████| 438M/438M [00:05<00:00, 76.4MB/s] 


In [4]:
# Define text input
text = 'hello world what a time to be alive!'

In [7]:
# Tokenize the sentence
tokens = tokenizer.encode_plus(text, 
                               max_length=128, 
                               truncation=True, 
                               padding='max_length', 
                               return_tensors='pt')

In [9]:
# Feed tokens
outputs = model(**tokens)

In [12]:
# Retrieve the embeddings of the sentence by taking the last layer output
embeddings = outputs.last_hidden_state